In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler


import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import timm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
LR = 1e-3
print(f"device = {device}")

device = cuda:0


In [3]:
class EnsembleModel(nn.Module):
    def __init__(self, num_classes):
        super(EnsembleModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.efficientnet = timm.create_model('efficientnet_b2', pretrained=True)
        self.vgg = models.vgg16(pretrained=True)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x1 = self.resnet(x)
        x2 = self.efficientnet(x)
        x3 = self.vgg(x)
        x = torch.cat((x1, x2, x3), dim=1)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 앙상블 모델 생성
num_classes = 19 # 분류할 클래스 수
model = EnsembleModel(num_classes)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\lee97/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 40.8MB/s]
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b2_ra-bcdf34b7.pth" to C:\Users\lee97/.cache\torch\hub\checkpoints\efficientnet_b2_ra-bcdf34b7.pth
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\lee97/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:14<00:00, 37.1MB/s] 


In [ ]:
# model = timm.create_model("resnet50", pretrained = True, num_classes = 19).to(device)

le = preprocessing.LabelEncoder()
optimizer = optim.Adam(model.parameters(), lr = LR)
scheduler = CosineAnnealingLR(optimizer, T_max = 32, eta_min = 1e-5)
label_map = {str(i): int(i) for i in range(19)}

In [4]:
CFG = {
    'IMG_SIZE':640,
    'EPOCHS':50,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':8,
    'SEED':41
}
EPOCH = CFG['EPOCHS']

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# dataset

In [5]:
all_img_list = glob.glob('data/train/*/*')

## if windows
for idx,image in enumerate(all_img_list):
    all_img_list[idx] = image.replace('\\', '/')

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        print(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
        

In [6]:
train_transform =  A.Compose([
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.2),
                            A.Rotate(limit=15),
                            A.Resize(440, 440),
                            A.OneOf([
                                A.GaussianBlur(blur_limit=3, p=0.2),
                                A.MotionBlur(blur_limit=3, p=0.2),
                                A.MedianBlur(blur_limit=3, p=0.1)
                            ], p=0.5),
                            A.CLAHE(clip_limit=2.0, p=0.2),
                            A.OneOf([
                                A.RandomBrightness(p=0.5),
                                A.RandomContrast(p=0.5),
                                A.RandomGamma(p=0.5),
                                A.HueSaturationValue(p=0.5)
                            ], p=0.5),
                            A.ISONoise(p=0.2),
                            A.ImageCompression(p=0.1),
                            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                            ToTensorV2()
])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [7]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [8]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in (val_loader):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            # labels = [[label_map[label] for label in tpl] for tpl in labels]
            # labels = torch.tensor(labels[0], dtype=torch.long).to(DEVICE)

            pred = model(imgs)
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score


def train(model, optimizer, train_loader, val_loader, scheduler):
    best_score = 0
    best_model = None
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    scaler = torch.cuda.amp.GradScaler()
    model.train()
    
    for epoch in (range(EPOCH)):
        batch = 0
        train_loss = []
        with tqdm(train_loader, unit='batch') as tepoch:
            for imgs, labels in tepoch:
                batch += 1
                optimizer.zero_grad()

                tepoch.set_description(f"Epoch {epoch}")
                imgs = imgs.float().to(DEVICE)
                labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
                # labels = [[label_map[label] for label in tpl] for tpl in labels]
                # labels = torch.tensor(labels[0], dtype=torch.long).to(DEVICE)
                
                with torch.cuda.amp.autocast():

                    output = model(imgs).to(DEVICE)
                    loss = criterion(output, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss.append(loss.item())

                tepoch.set_postfix(
                    phase="Training",
                    loss=loss.item()
                )

            _val_loss, _val_score = validation(model, criterion, val_loader, DEVICE)
            _train_loss = np.mean(train_loss)
            print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')

            scheduler.step()
                    
            if best_score < _val_score:
                best_score = _val_score
                best_model = model
                print("Best_model saved")
    return best_model


def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm((test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [9]:
infer_model = train(model, optimizer, train_loader, val_loader, scheduler)

  0%|          | 0/303 [00:00<?, ?batch/s]

data/train/훼손/878.png


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
preds = inference(infer_model, test_loader, DEVICE)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False, encoding='utf-8')